In [227]:
import pandas as pd
import seaborn as sb
import numpy as np
import decimal
import math as m
from matplotlib import pyplot as plt

# Reader score must be set to a very small value otherwise there will be a division by 0

epsilon = 0.000001

# CSV file parsing

dataset_name = "seed_1"
seed_folder_path = f"data/seed/{dataset_name}/"
info_filename = f"{seed_folder_path}info.csv"
ratings_filename = f"{seed_folder_path}ratings.csv"
authors_filename = f"{seed_folder_path}authors.csv"

info = pd.read_csv(info_filename)
paper_authors = pd.read_csv(authors_filename)
paper_authors = paper_authors.values
paper_ratings = pd.read_csv(ratings_filename)
paper_ratings = paper_ratings.values

# Initial setup

dataset_name = info["Dataset"][0]
papers_number = info["Paper"][0]
readers_number = info["Reader"][0]
ratings_number = info["Rating"][0]
authors_number = info["Author"][0]
papers = np.arange(papers_number)
readers = np.arange(readers_number)
ratings = np.arange(ratings_number)
authors = np.arange(authors_number)
paper_steadiness = np.zeros(papers_number,dtype=np.dtype(decimal.Decimal))
paper_score = np.zeros(papers_number,dtype=np.dtype(decimal.Decimal))
rating_goodness = np.zeros(ratings_number,dtype=np.dtype(decimal.Decimal))
reader_steadiness = np.zeros(readers_number,dtype=np.dtype(decimal.Decimal))
reader_score = np.zeros(readers_number,dtype=np.dtype(decimal.Decimal))
reader_score.fill(epsilon)
author_steadiness = np.zeros(authors_number, dtype=np.dtype(decimal.Decimal))
author_score = np.zeros(authors_number, dtype=np.dtype(decimal.Decimal))

def get_author(current_paper) :
    found_authors = []
    for author_index, author_entry in enumerate(paper_authors) :
        current_author = int(author_entry[0])
        written_papers = author_entry[1].split(";")
        written_papers = [int(x) for x in written_papers]
        if current_paper in written_papers :
            found_authors.append(current_author)
    return np.asarray(found_authors)


for index, entry in enumerate(paper_ratings) :

    # Example: <1,1,2,0.8,0>
    # At Timestamp 1 Reader 1 gave to Paper 2 a Rating of 0.8 written by Author 0
    timestamp = int(entry[0])
    reader = int(entry[1])
    paper = int(entry[2])
    rating = entry[3]
    authors_of_paper = get_author(paper)

    print(f"TIMESTAMP {timestamp} - READER {reader} - PAPER {paper} - SCORE {rating}")

    # COMPUTATION START: PAPER AND READER SCORE

    # Saving values at time t(i)

    old_paper_steadiness = paper_steadiness[paper]
    old_paper_score = paper_score[paper]
    old_reader_steadiness = reader_steadiness[reader]
    old_rating_goodness = rating_goodness[timestamp]
    old_reader_score = reader_score[reader]

    # Updating values at time t(i+1)

    paper_steadiness[paper] = round((old_paper_steadiness + old_reader_score), 16)
    paper_score[paper] = round((((old_paper_steadiness * old_paper_score) + (old_reader_score * rating)) / paper_steadiness[paper]), 16)
    rating_goodness[timestamp] = round((1 - (m.sqrt(abs(rating - paper_score[paper])))), 16)
    reader_steadiness[reader] = round((old_reader_steadiness + paper_steadiness[paper]),16)
    reader_score[reader] = round((((old_reader_steadiness * old_reader_score) + (paper_steadiness[paper] * rating_goodness[timestamp])) / reader_steadiness[reader]), 16)

    # COMPUTATION START: AUTHOR SCORE

    for author in authors_of_paper :
        # Saving values at time t(i)

        old_author_steadiness = author_steadiness[author]
        old_author_score = author_score[author]

        # Updating values at time t(i+1)

        author_steadiness[author] = old_author_steadiness + old_reader_score
        author_score[author] = ((old_author_steadiness * old_author_score) + (old_reader_score * rating)) / author_steadiness[author]

    # COMPUTATION START: PROPAGATING CHANGES TO PREVIOUS READERS

    if index > 0 :

        previous_paper_ratings = pd.DataFrame(
            paper_ratings[:index],
            columns=["Timestamp", "Reader", "Paper", "Score", "Author"]
        )
        previous_paper_ratings["Score"] = previous_paper_ratings["Score"]
        previous_paper_ratings = previous_paper_ratings.loc[
            (previous_paper_ratings["Paper"] == float(paper)) &
            (previous_paper_ratings["Reader"] != float(reader))
            ]

        print(previous_paper_ratings)
        previous_paper_ratings = previous_paper_ratings.values

        print(" ----- PREVIOUS PAPER RATINGS -----")

        for previous_index, previous_entry in enumerate(previous_paper_ratings):
            
            # Example: <1,1,2,0.8,0>
            # At Timestamp 1 Reader 1 gave to Paper 2 a Rating of 0.8 written by Author 0
            previous_timestamp = int(previous_entry[0])
            previous_reader = int(previous_entry[1])
            previous_paper = int(previous_entry[2])
            previous_rating = previous_entry[3]

            print(f"PREVIOUS TIMESTAMP {previous_timestamp} - PREVIOUS READER {previous_reader} - PREVIOUS PAPER {previous_paper} - PREVIOUS RATING {previous_rating}")

            # Saving previous values at time t(i)

            old_previous_reader_steadiness = reader_steadiness[previous_reader]
            old_previous_reader_score = reader_score[previous_reader]
            old_previous_rating = previous_rating
            old_previous_rating_goodness = rating_goodness[previous_timestamp]

            # Updating previous values at time t(i+1)

            rating_goodness[previous_timestamp] = 1 - (m.sqrt(abs(old_previous_rating - paper_score[paper])))
            reader_steadiness[previous_reader] = (old_previous_reader_steadiness + old_reader_score)
            reader_score[previous_reader] = (
                                                    (old_previous_reader_steadiness * old_previous_reader_score) -
                                                    (old_paper_steadiness * old_previous_rating_goodness) +
                                                    (paper_steadiness[paper] * rating_goodness[previous_timestamp])
                                            ) / reader_steadiness[previous_reader]

        print(" ----- PREVIOUS PAPER RATINGS END -----")

print("-------")


TIMESTAMP 0 - READER 0 - PAPER 0 - SCORE 0.8164039671955987
TIMESTAMP 1 - READER 0 - PAPER 1 - SCORE 0.5257531140469901
Empty DataFrame
Columns: [Timestamp, Reader, Paper, Score, Author]
Index: []
 ----- PREVIOUS PAPER RATINGS -----
 ----- PREVIOUS PAPER RATINGS END -----
TIMESTAMP 2 - READER 1 - PAPER 0 - SCORE 0.4894699616853298
   Timestamp  Reader  Paper     Score  Author
0        0.0     0.0    0.0  0.816404     1.0
 ----- PREVIOUS PAPER RATINGS -----
PREVIOUS TIMESTAMP 0 - PREVIOUS READER 0 - PREVIOUS PAPER 0 - PREVIOUS RATING 0.8164039671955987
 ----- PREVIOUS PAPER RATINGS END -----
TIMESTAMP 3 - READER 1 - PAPER 1 - SCORE 0.1280936045479151
   Timestamp  Reader  Paper     Score  Author
1        1.0     0.0    1.0  0.525753     1.0
 ----- PREVIOUS PAPER RATINGS -----
PREVIOUS TIMESTAMP 1 - PREVIOUS READER 0 - PREVIOUS PAPER 1 - PREVIOUS RATING 0.5257531140469901
 ----- PREVIOUS PAPER RATINGS END -----
TIMESTAMP 4 - READER 2 - PAPER 0 - SCORE 0.4621439549371218
   Timestamp  Rea


 ----- PREVIOUS PAPER RATINGS -----
PREVIOUS TIMESTAMP 1 - PREVIOUS READER 0 - PREVIOUS PAPER 1 - PREVIOUS RATING 0.5257531140469901
PREVIOUS TIMESTAMP 3 - PREVIOUS READER 1 - PREVIOUS PAPER 1 - PREVIOUS RATING 0.1280936045479151
PREVIOUS TIMESTAMP 5 - PREVIOUS READER 2 - PREVIOUS PAPER 1 - PREVIOUS RATING 0.13924398266236113
PREVIOUS TIMESTAMP 7 - PREVIOUS READER 3 - PREVIOUS PAPER 1 - PREVIOUS RATING 0.05329445422744145
PREVIOUS TIMESTAMP 9 - PREVIOUS READER 4 - PREVIOUS PAPER 1 - PREVIOUS RATING 0.2858043401987116
PREVIOUS TIMESTAMP 11 - PREVIOUS READER 5 - PREVIOUS PAPER 1 - PREVIOUS RATING 0.6683366936862787
PREVIOUS TIMESTAMP 13 - PREVIOUS READER 6 - PREVIOUS PAPER 1 - PREVIOUS RATING 0.12806297672895176
 ----- PREVIOUS PAPER RATINGS END -----
TIMESTAMP 16 - READER 8 - PAPER 0 - SCORE 0.4792847492393352
    Timestamp  Reader  Paper     Score  Author
0         0.0     0.0    0.0  0.816404     1.0
2         2.0     1.0    0.0  0.489470     2.0
4         4.0     2.0    0.0  0.46214


 ----- PREVIOUS PAPER RATINGS -----
PREVIOUS TIMESTAMP 0 - PREVIOUS READER 0 - PREVIOUS PAPER 0 - PREVIOUS RATING 0.8164039671955987
PREVIOUS TIMESTAMP 2 - PREVIOUS READER 1 - PREVIOUS PAPER 0 - PREVIOUS RATING 0.4894699616853298
PREVIOUS TIMESTAMP 4 - PREVIOUS READER 2 - PREVIOUS PAPER 0 - PREVIOUS RATING 0.4621439549371218
PREVIOUS TIMESTAMP 6 - PREVIOUS READER 3 - PREVIOUS PAPER 0 - PREVIOUS RATING 0.7754196550844624
PREVIOUS TIMESTAMP 8 - PREVIOUS READER 4 - PREVIOUS PAPER 0 - PREVIOUS RATING 0.06947407922406873
PREVIOUS TIMESTAMP 10 - PREVIOUS READER 5 - PREVIOUS PAPER 0 - PREVIOUS RATING 0.3447313369760282
PREVIOUS TIMESTAMP 12 - PREVIOUS READER 6 - PREVIOUS PAPER 0 - PREVIOUS RATING 0.5705509987308046
PREVIOUS TIMESTAMP 14 - PREVIOUS READER 7 - PREVIOUS PAPER 0 - PREVIOUS RATING 0.5114046931132381
PREVIOUS TIMESTAMP 16 - PREVIOUS READER 8 - PREVIOUS PAPER 0 - PREVIOUS RATING 0.4792847492393352
PREVIOUS TIMESTAMP 18 - PREVIOUS READER 9 - PREVIOUS PAPER 0 - PREVIOUS RATING 0.3273

In [228]:
# Summary

print("PAPER SCORE:       ", paper_score)
print("PAPER STEADINESS:  ", paper_steadiness)
print("READER SCORE:      ", reader_score)
print("READER STEADINESS: ", reader_steadiness)
print("AUTHOR SCORE:      ", author_score)
print("AUTHOR STEADINESS: ", author_steadiness)

PAPER SCORE:        [0.4963417367684596 0.4048518775828114]
PAPER STEADINESS:   [2e-05 12.830517122033056]
READER SCORE:       [0.6522911090991149 0.47392249032991846 0.4846289065213337
 0.40707732050318945 0.6549668580695441 0.48669247251504855
 0.4738930861955109 0.3851495352897483 0.3034347451926476
 0.4851089389645505 0.39848831273618424 0.6320280007631598
 0.694189599816185 0.5709059710290627 0.5820341049472865 0.58262304172089
 0.509287789665014 0.4117439320111684 0.8876016196049921
 0.3406108701115647]
READER STEADINESS:  [12.830537122033052 12.830537122033052 12.830537122033052
 12.830537122033052 12.830537122033052 12.830537122033052
 12.830537122033052 12.830537122033052 12.830537122033052
 12.83053712203305 12.83053712203305 12.83053712203305 12.83053712203305
 12.830537122033052 12.830537122033052 12.830537122033054
 12.830537122033054 12.830537122033055 12.830537122033055
 12.830537122033055]
AUTHOR SCORE:       [0.4048520201954833 0.4048520201954833 0.4048520201954833
 0.